In [1]:
!pip install -U datasets
!pip install fsspec==2023.9.2
!pip install -U transformers datasets peft accelerate --quiet

In [1]:
pip install wandb


In [3]:
!wandb login

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nain21 (nain21-iit-kharagpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
!pip install datasets transformers peft accelerate wandb fastapi uvicorn



In [3]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [8]:
import transformers
print(transformers.__version__)


4.52.3


In [11]:
import torch
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from peft import LoraConfig, get_peft_model
import wandb

# Optional: Initialize wandb
# wandb.init(project="bart-lora-multiwoz", entity="nain21")

# Load multi_woz_v22 dataset
dataset = load_dataset("multi_woz_v22", cache_dir="/tmp/hf_cache")

# Load tokenizer and model
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Should show ~1.14% trainable parameters

# Preprocessing
max_input_length = 512
max_target_length = 128

def preprocess_function(example):
    dialogue = " ".join([turn["text"] for turn in example["dialogue"]])
    summary = example["dialogue"][-1]["text"]  # Use last utterance as pseudo-summary

    model_inputs = tokenizer(dialogue, max_length=max_input_length, padding="max_length", truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(summary, max_length=max_target_length, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def filter_fn(example):
    return "dialogue" in example and len(example["dialogue"]) > 1

# Apply preprocessing
train_dataset = dataset["train"].filter(filter_fn).map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
val_dataset = dataset["validation"].filter(filter_fn).map(preprocess_function, batched=True, remove_columns=dataset["validation"].column_names)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./bart-lora-multiwoz",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-4,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=50,
    # report_to="wandb",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save LoRA adapter
model.save_pretrained("./bart-lora-multiwoz-adapter")

# Push to Hugging Face Hub
model.push_to_hub("levi121/bart-large-cnn-lora-multiwoz-adapter", private=False)


trainable params: 1,179,648 || all params: 407,470,080 || trainable%: 0.2895


<ipython-input-11-2973e3eb8a59>:75: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


ValueError: No columns in the dataset match the model's forward method signature: ({', '.join(signature_columns)}). The following columns have been ignored: []. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [13]:
import torch
from datasets import load_dataset
from transformers import (
    BartTokenizer,
    BartForConditionalGeneration,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from peft import LoraConfig, get_peft_model
import wandb

# Optional: Initialize wandb
# wandb.init(project="bart-lora-multiwoz", entity="nain21")

# Load multi_woz_v22 dataset (with trust_remote_code if needed)
dataset = load_dataset("multi_woz_v22", cache_dir="/tmp/hf_cache", trust_remote_code=True)

# Load tokenizer and model
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# LoRA configuration (corrected target modules for BART)
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "self_attn.q_proj", "self_attn.v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Preprocessing
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    # Process turns (using "utterance" in each turn)
    dialogues = [
        " ".join([turn["utterance"] for turn in dialogue_turns])
        for dialogue_turns in examples["turns"]
    ]
    # Use last turn's utterance as pseudo-summary
    summaries = [dialogue_turns[-1]["utterance"] for dialogue_turns in examples["turns"]]

    # Tokenize inputs
    model_inputs = tokenizer(
        dialogues,
        max_length=max_input_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            summaries,
            max_length=max_target_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def filter_fn(example):
    # Filter out single-turn dialogues
    return len(example["turns"]) > 1

# Apply preprocessing with correct column removal
train_dataset = dataset["train"].filter(filter_fn).map(
    preprocess_function,
    batched=True,
    remove_columns=["turns", "dialogue_id", "services"]  # Remove actual existing columns
)
val_dataset = dataset["validation"].filter(filter_fn).map(
    preprocess_function,
    batched=True,
    remove_columns=["turns", "dialogue_id", "services"]
)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./bart-lora-multiwoz",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-4,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=50,
    # report_to="wandb",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    remove_unused_columns=False  # Safety measure
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save LoRA adapter
model.save_pretrained("./bart-lora-multiwoz-adapter")

# Push to Hugging Face Hub
model.push_to_hub("levi121/bart-large-cnn-lora-multiwoz-adapter", private=False)


trainable params: 1,179,648 || all params: 407,470,080 || trainable%: 0.2895


Filter:   0%|          | 0/8437 [00:00<?, ? examples/s]

Map:   0%|          | 0/8437 [00:00<?, ? examples/s]

TypeError: string indices must be integers, not 'str'

In [4]:
from transformers import BartTokenizer, BartForConditionalGeneration
from peft import PeftModel

model_name = "facebook/bart-large-cnn"
adapter_name = "levi121/bart-large-cnn-lora-samsum-adapter"

tokenizer = BartTokenizer.from_pretrained(model_name)
base_model = BartForConditionalGeneration.from_pretrained(model_name)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, adapter_name)
model.eval()

def summarize(text):
    inputs = tokenizer(text, max_length=512, truncation=True, return_tensors="pt")
    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=4,
            early_stopping=True
        )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
dialogue = "John: Hey, are we still meeting tomorrow?\nJane: Yes, 10am at the cafe."
print(summarize(dialogue))


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

None of the available devices `available_devices = None` are supported by the bitsandbytes version you have installed: `bnb_supported_devices = {'mps', 'hpu', '"cpu" (needs an Intel CPU and intel_extension_for_pytorch installed and compatible with the PyTorch version)', 'cuda', 'xpu', 'npu'}`. Please check the docs to see if the backend you intend to use is available and how to install it: https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: None of the available devices `available_devices = None` are supported by the bitsandbytes version you have installed: `bnb_supported_devices = {'mps', 'hpu', '"cpu" (needs an Intel CPU and intel_extension_for_pytorch installed and compatible with the PyTorch version)', 'cuda', 'xpu', 'npu'}`. Please check the docs to see if the backend you intend to use is available and how to install it: https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend